# Background

In HW1, we examined codon frequencies in the COVID genome.
While the purpose of that exercise was mostly for getting familiar with processing genomic data, analyzing codon frequencies is a useful, albeit low resolution, method to gain insight into how an organism's genetic code impacts its molecular and cellular processes (read more about [codon usage bias](https://en.wikipedia.org/wiki/Codon_usage_bia)).

More generally, [k-mers](https://en.wikipedia.org/wiki/K-mer) are a quick but powerful way to generate a signature of a sequence (under the lense of k-mers, codons can also be viewed as non-overlapping 3-mers).
K-mer frequencies of a genome can be used as a "molecular fingerprint" of a species.

![k-mer](https://miro.medium.com/v2/resize:fit:1400/1*YLSWj-6ktXiYmK5zHdPwWg.png)

[VirFinder](https://doi.org/10.1186/s40168-017-0283-5) is an example of a computational tool that leverages k-mer frequencies to sniff out these fingerprints.
VirFinder classifies sequences as belonging to a prokaryote (bacteria and archaea) or to a virus, purely using k-mer frequencies.
This is useful, because viruses, unlike cellular organisms, do not share any conserved marker genes (between viral families).
In fact, viruses are so phylogenetically diverse that some researchers hypothesize that viruses do not even share a common ancestor, i.e. they emerged independently!
Because viruses are so diverse, gene-based similarity approaches may fail to detect viruses that are only distantly related to those in current viral genome databases.
Thus, k-mer frequencies can be a powerful tool to analyze the viral content of a [metagenomic sample](https://en.wikipedia.org/wiki/Metagenomics) (you can abstract this as simply a bunch of fragments of DNA from all the microorganisms in an environment) of a complex microbial environment.

![virfinder](https://raw.githubusercontent.com/danielchang2002/5481_supplementary/main/virfinder.png)

This notebook will demonstrate how you can use easily use k-mer statistics to build your own machine learning model to classify sequences as viral or magic cow.
Essentially, in this notebook, we will build out a variant of VirFinder from scratch!
Except ours will be kind of trash 💩, because it will be only trained on one species per class, and will not be very useful (what is useful is identifying viral sequences from a sea of bacterial DNA).
But it will hopefully be fun!


# The problem

Let's say you are a chocolate milk farmer.
That is, you have a herd of magic cows that produce chocolate milk. Oh no, your cows have been getting sick!
This is sad, because your sick magic cows only produce normal cow milk :(

![cow](https://media.tenor.com/T2knfgjNvcsAAAAC/the-rock-cow.gif)

You suspect they have the seasonal moo moo flu, but PCR and antigen tests on the sick cows return negative results.
So naturally, you take some milk from your cows and run it through the Illumina sequencer that's just sitting in your barn.

![sequencer](https://www.illumina.com/content/dam/illumina-marketing/images/systems/v2/table-system-thumbs/table-graphic-system-miseq.png)

Now that you have the magic cow milk sequences (for simplicity, assume the sample contains both magic cow DNA and the viral DNA, but nothing else), you just need to identify the virus, order drugs from Big Farma, and cure your cows!
However, after throwing the sequences into [BLAST](https://blast.ncbi.nlm.nih.gov/Blast.cgi), you get no results.
The cows must be infected with some novel undiscovered virus.
Makes sense I guess, since they are magic cows.

You have the idea of simply identifying and discarding the sequences that belong to the magic cows, leaving you with just the viral sequences, but because magic cows are a government secret, their reference genome has not been published.

Fortunately, you are taking CSCI 5481!
You recall that k-mer frequencies are distinct between different branches of life.
Thus, if we can somehow model the differences in k-mer frequencies between mammalian genomes and viral genomes (which should be sufficiently different at certain choices of k), we can pick out the viral sequences from the milk samples.
Again, this can all be done without knowing the genomes of the magic cow and the magic cow virus!

Below, we will extract k-mer frequencies from fragments of the (normal) cow genome and the covid genome, and build a simple machine learning model to classify these fragments.
We will then apply this model to fragments of DNA from the magic cow and the magic virus (obviously these are both imaginary species, we will use the platypus genome and the herpes virus genome).
We will evaluate the performance of the model and see if your herd can be saved.

# Let's download and prepare the data!

In [1]:
# Download the cow genome
! curl 'https://api.ncbi.nlm.nih.gov/datasets/v2alpha/genome/download?filename=GCF_002263795.2.zip&ncbi_phid=322C40369303251500003229588A93FC.1.m_6.044' \
  --data-raw 'arr__accessions=GCF_002263795.2&arr__include_annotation_type=GENOME_FASTA&include_tsv=true&action=https%3A%2F%2Fapi.ncbi.nlm.nih.gov%2Fdatasets%2Fv2alpha%2Fgenome%2Fdownload%3Ffilename%3DGCF_002263795.2.zip%26ncbi_phid%3D322C40369303251500003229588A93FC.1.m_6.044&x-datasets-client=datasets&x-datasets-client-page=taxonomy_record' \
  --compressed \
  --output cow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  801M    0  801M  100   329  22.5M      9  0:00:36  0:00:35  0:00:01 23.1M


In [2]:
# Download the covid genome
! curl 'https://api.ncbi.nlm.nih.gov/datasets/v2alpha/genome/download?filename=GCF_009858895.2.zip&ncbi_phid=322C6EA4A63DE9D500005260D41BFD0E.1.m_7.037' \
  --data-raw 'arr__accessions=GCF_009858895.2&arr__include_annotation_type=GENOME_FASTA&include_tsv=true&action=https%3A%2F%2Fapi.ncbi.nlm.nih.gov%2Fdatasets%2Fv2alpha%2Fgenome%2Fdownload%3Ffilename%3DGCF_009858895.2.zip%26ncbi_phid%3D322C6EA4A63DE9D500005260D41BFD0E.1.m_7.037&x-datasets-client=datasets&x-datasets-client-page=genome_record' \
  --compressed \
  --output covid.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12513    0 12186  100   327  21489    576 --:--:-- --:--:-- --:--:-- 22068


In [3]:
# download magic cow genome (actually the platypus genome, because the platypus is a magical looking animal)
! curl 'https://api.ncbi.nlm.nih.gov/datasets/v2alpha/genome/download?filename=GCF_004115215.2.zip&ncbi_phid=939B746959BF6E1500005160F1A309E8.1.m_5.026' \
  --data-raw 'arr__accessions=GCF_004115215.2&arr__include_annotation_type=GENOME_FASTA&include_tsv=true&action=https%3A%2F%2Fapi.ncbi.nlm.nih.gov%2Fdatasets%2Fv2alpha%2Fgenome%2Fdownload%3Ffilename%3DGCF_004115215.2.zip%26ncbi_phid%3D939B746959BF6E1500005160F1A309E8.1.m_5.026&x-datasets-client=datasets&x-datasets-client-page=taxonomy_record' \
  --compressed \
  --output magic_cow.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  545M    0  545M  100   329  25.5M     15  0:00:21  0:00:21 --:--:-- 27.0M


In [4]:
# download magic cow virus genome (actually herpes)
! curl 'https://api.ncbi.nlm.nih.gov/datasets/v2alpha/genome/download?filename=GCF_000859985.2.zip&ncbi_phid=939B746959BF6E15000059620EF36C5C.1.m_7.045' \
  --data-raw 'arr__accessions=GCF_000859985.2&arr__include_annotation_type=GENOME_FASTA&include_tsv=true&action=https%3A%2F%2Fapi.ncbi.nlm.nih.gov%2Fdatasets%2Fv2alpha%2Fgenome%2Fdownload%3Ffilename%3DGCF_000859985.2.zip%26ncbi_phid%3D939B746959BF6E15000059620EF36C5C.1.m_7.045&x-datasets-client=datasets&x-datasets-client-page=genome_record' \
  --compressed \
  --output magic_cow_virus.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 47691    0 47364  100   327   148k   1048 --:--:-- --:--:-- --:--:--  149k


In [5]:
# unzip stuff
! unzip -d cow cow.zip
! unzip -d covid covid.zip
! unzip -d magic_cow magic_cow.zip
! unzip -d magic_cow_virus magic_cow_virus.zip

Archive:  cow.zip
  inflating: cow/README.md           
  inflating: cow/ncbi_dataset/data/data_summary.tsv  
  inflating: cow/ncbi_dataset/data/assembly_data_report.jsonl  
  inflating: cow/ncbi_dataset/data/GCF_002263795.2/GCF_002263795.2_ARS-UCD1.3_genomic.fna  
  inflating: cow/ncbi_dataset/data/dataset_catalog.json  
Archive:  covid.zip
  inflating: covid/README.md         
  inflating: covid/ncbi_dataset/data/data_summary.tsv  
  inflating: covid/ncbi_dataset/data/assembly_data_report.jsonl  
  inflating: covid/ncbi_dataset/data/GCF_009858895.2/GCF_009858895.2_ASM985889v3_genomic.fna  
  inflating: covid/ncbi_dataset/data/dataset_catalog.json  
Archive:  magic_cow.zip
  inflating: magic_cow/README.md     
  inflating: magic_cow/ncbi_dataset/data/data_summary.tsv  
  inflating: magic_cow/ncbi_dataset/data/assembly_data_report.jsonl  
  inflating: magic_cow/ncbi_dataset/data/GCF_004115215.2/GCF_004115215.2_mOrnAna1.pri.v4_genomic.fna  
  inflating: magic_cow/ncbi_dataset/data/datas

In [6]:
# let's remove line wrapping and rename files
! awk '/^>/ {printf("\n%s\n",$0);next; } { printf("%s",$0);}  END {printf("\n");}' < /content/cow/ncbi_dataset/data/GCF_002263795.2/GCF_002263795.2_ARS-UCD1.3_genomic.fna | sed '/^$/d' > cow.fna
! awk '/^>/ {printf("\n%s\n",$0);next; } { printf("%s",$0);}  END {printf("\n");}' < /content/covid/ncbi_dataset/data/GCF_009858895.2/GCF_009858895.2_ASM985889v3_genomic.fna | sed '/^$/d' > covid.fna
! awk '/^>/ {printf("\n%s\n",$0);next; } { printf("%s",$0);}  END {printf("\n");}' < /content/magic_cow/ncbi_dataset/data/GCF_004115215.2/GCF_004115215.2_mOrnAna1.pri.v4_genomic.fna | sed '/^$/d' > magic_cow.fna
! awk '/^>/ {printf("\n%s\n",$0);next; } { printf("%s",$0);}  END {printf("\n");}' < /content/magic_cow_virus/ncbi_dataset/data/GCF_000859985.2/GCF_000859985.2_ViralProj15217_genomic.fna | sed '/^$/d' > magic_cow_virus.fna

# Let's explore the data!

In [7]:
# Take a look at the first few headers in the cow fasta file
! grep ">" cow.fna -m 35

>NC_037328.1 Bos taurus isolate L1 Dominette 01449 registration number 42190680 breed Hereford chromosome 1, ARS-UCD1.2, whole genome shotgun sequence
>NC_037329.1 Bos taurus isolate L1 Dominette 01449 registration number 42190680 breed Hereford chromosome 2, ARS-UCD1.2, whole genome shotgun sequence
>NC_037330.1 Bos taurus isolate L1 Dominette 01449 registration number 42190680 breed Hereford chromosome 3, ARS-UCD1.2, whole genome shotgun sequence
>NC_037331.1 Bos taurus isolate L1 Dominette 01449 registration number 42190680 breed Hereford chromosome 4, ARS-UCD1.2, whole genome shotgun sequence
>NC_037332.1 Bos taurus isolate L1 Dominette 01449 registration number 42190680 breed Hereford chromosome 5, ARS-UCD1.2, whole genome shotgun sequence
>NC_037333.1 Bos taurus isolate L1 Dominette 01449 registration number 42190680 breed Hereford chromosome 6, ARS-UCD1.2, whole genome shotgun sequence
>NC_037334.1 Bos taurus isolate L1 Dominette 01449 registration number 42190680 breed Hereford

In [8]:
! grep ">" covid.fna

>NC_045512.2 Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome


### Let's just work with the first cow chromosome

In [9]:
with open("cow.fna") as f:
  header = f.readline()
  cow = f.readline()

In [10]:
print(header)
print("Chromosome 1 assembly has", f"{len(cow):,}", "base pairs")
bases_per_line = 8000
for i in range(40):
  print(cow[i * bases_per_line: (i + 1) * bases_per_line])
print("...")

>NC_037328.1 Bos taurus isolate L1 Dominette 01449 registration number 42190680 breed Hereford chromosome 1, ARS-UCD1.2, whole genome shotgun sequence

Chromosome 1 assembly has 158,534,111 base pairs
...
...


### Question: Why are there lowercase and upper cases ACTGs?
### Answer from ChatGPT:

Upper Case Letters (Capital Letters):

Coding Regions (Exons): In the genome, coding regions called exons are made up of sequences of DNA that are transcribed into messenger RNA (mRNA) and ultimately translated into proteins. These sequences are typically represented using uppercase letters. Uppercase letters indicate regions of the genome that encode functional proteins.

Conserved Sequences: Some genomic regions are highly conserved across species, and these conserved sequences are usually represented in uppercase letters. Conservation suggests that these sequences have an essential biological function.

Lower Case Letters:

Non-Coding Regions (Introns, Intergenic Regions, and Non-Coding RNA): The majority of the genome consists of non-coding regions, which do not encode proteins. This includes introns within genes, intergenic regions between genes, and sequences that serve as regulatory elements or non-coding RNA genes. Lowercase letters are often used to represent these non-coding regions. This distinction helps researchers identify functional coding regions from non-coding ones.

Variability and Uncertainty: Lowercase letters can also be used to indicate regions of uncertainty, where sequencing or assembly quality is lower. These regions may contain gaps, repetitive sequences, or regions that are difficult to assemble accurately.
asdf

### Let's load up the covid genome

In [11]:
with open("covid.fna") as f:
  header = f.readline()
  covid = f.readline()

In [12]:
print(header)
print("COVID genome has", f"{len(covid):,}", "base pairs")
bases_per_line = 100
for i in range(20):
  print(covid[i * bases_per_line: (i + 1) * bases_per_line])
print("...")

>NC_045512.2 Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome

COVID genome has 29,904 base pairs
ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC
GGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTACTGTCGTTGACAGGACACGAGTAACTCGTCTATCTTCTGCAGGCTGCTTACGGT
TTCGTCCGTGTTGCAGCCGATCATCAGCACATCTAGGTTTCGTCCGGGTGTGACCGAAAGGTAAGATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAAC
ACACGTCCAACTCAGTTTGCCTGTTTTACAGGTTCGCGACGTGCTCGTACGTGGCTTTGGAGACTCCGTGGAGGAGGTCTTATCAGAGGCACGTCAACAT
CTTAAAGATGGCACTTGTGGCTTAGTAGAAGTTGAAAAAGGCGTTTTGCCTCAACTTGAACAGCCCTATGTGTTCATCAAACGTTCGGATGCTCGAACTG
CACCTCATGGTCATGTTATGGTTGAGCTGGTAGCAGAACTCGAAGGCATTCAGTACGGTCGTAGTGGTGAGACACTTGGTGTCCTTGTCCCTCATGTGGG
CGAAATACCAGTGGCTTACCGCAAGGTTCTTCTTCGTAAGAACGGTAATAAAGGAGCTGGTGGCCATAGTTACGGCGCCGATCTAAAGTCATTTGACTTA
GGCGACGAGCTTGGCACTGATCCTTATGAAGATTTTCAAGAAAACTGGAACACTAAACATAGCAGTGGTGTTACCCGTGAACTCATGCGTGAGCTTAACG
GAGGGGCATACACTCGCTATGTCGATAACAACTTCTGTGGCCCTGATGGCTACCCTCTT

### Let's do the same for the test data

In [13]:
with open("magic_cow.fna") as f:
  header = f.readline()
  magic_cow = f.readline()

with open("magic_cow_virus.fna") as f:
  header = f.readline()
  magic_cow_virus = f.readline()

# Let's create a simulated dataset!

Let's take 100 disjoint substrings of all the genomes, each 250 bases long

In [14]:
length = 250
offset = 200000 # to avoid repetitive regions in the beginning of cow genome

cow_substrings = []
covid_substrings = []
magic_cow_substrings = []
magic_cow_virus_substrings = []

for i in range(100):
  start = i * length
  end = (i + 1) * length
  cow_substrings.append(cow[offset + start: offset + end].upper())
  covid_substrings.append(covid[start:end])
  magic_cow_substrings.append(magic_cow[offset + start: offset + end].upper())
  magic_cow_virus_substrings.append(magic_cow_virus[start:end])

These substrings simulate [contigs](https://en.wikipedia.org/wiki/Contig) from an assembled metagenome

In [15]:
covid_substrings[-1]

'GCATGTGACTTATGTCCCTGCACAAGAAAAGAACTTCACAACTGCTCCTGCCATTTGTCATGATGGAAAAGCACACTTTCCTCGTGAAGGTGTCTTTGTTTCAAATGGCACACACTGGTTTGTAACACAAAGGAATTTTTATGAACCACAAATCATTACTACAGACAACACATTTGTGTCTGGTAACTGTGATGTTGTAATAGGAATTGTCAACAACACAGTTTATGATCCTTTGCAACCTGAATTAGAC'

In [16]:
import numpy as np

# let viruses be 1, and non-virus be 0
train_labels = np.array([0] * 100 + [1] * 100)
train_strings = cow_substrings + covid_substrings

### TODO: finish this function! Turn the strings into k-mer frequencies

In [17]:
from collections import defaultdict
def count_kmers(s, k=2):

  counts = defaultdict(int)

  #-----------------------------Start code here---------------------------------

  # Go through s and count the k-mers!
  # i.e, start with characters 1 to k, then characters 2 to (k + 1)... (overlapping)
  # note, this is different from the hw, where you start with characters 1-3, then characters 4-6 (non-overlapping)

  #-----------------------------End code here---------------------------------

  return counts

In [18]:
# quick test to see if your code matches my result
assert(count_kmers(train_strings[0], k=2) == {'CA': 30,
             'AC': 34,
             'CT': 38,
             'TG': 11,
             'GC': 4,
             'CC': 23,
             'TT': 18,
             'TC': 30,
             'TA': 15,
             'AG': 4,
             'GT': 4,
             'GG': 4,
             'GA': 7,
             'AT': 14,
             'AA': 12}
)

In [19]:
from itertools import product
import numpy as np

def dict_to_array(counts, k=2):
  possible_kmers = [''.join(li) for li in list(product("ACTG", repeat=k))]
  kmer_to_index = {kmer : i for i, kmer in enumerate(possible_kmers)}
  count_array = np.zeros(len(possible_kmers))
  for kmer in counts:
    count = counts[kmer]
    count_array[kmer_to_index[kmer]] = count
  return count_array

In [20]:
train_kmer_frequencies = np.stack([dict_to_array(count_kmers(s)) for s in train_strings])
print(train_kmer_frequencies[0])
train_kmer_frequencies.shape

[12. 34. 14.  4. 30. 23. 38.  0. 15. 30. 18. 11.  7.  4.  4.  4.]


(200, 16)

# Let's train the model!

In [21]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(penalty="l1", solver='liblinear')

model.fit(train_kmer_frequencies, train_labels)

LogisticRegression(penalty='l1', solver='liblinear')

# Let's perform inference on the test data

In [22]:
test_labels = np.array([0] * 100 + [1] * 100)
test_strings = magic_cow_substrings + magic_cow_virus_substrings

test_kmer_frequencies = np.stack([dict_to_array(count_kmers(s)) for s in test_strings])
print(test_kmer_frequencies[0])
test_kmer_frequencies.shape

[31. 21. 10. 20. 19. 12. 15.  4. 16.  5. 22. 17. 17. 11. 13. 15.]


(200, 16)

In [23]:
from sklearn.metrics import accuracy_score

pred = model.predict(test_kmer_frequencies)

accuracy_score(pred, test_labels)

0.635

# Excercises

Well we only got 63.5% accuracy on classifying sequences as viral or not.
So your cows might be doomed.

![cow](https://i.kym-cdn.com/entries/icons/original/000/028/626/cover3.jpg)

However, our low resolution methodology leaves much to be desired.
Below are some exercises to try out to potentially save the cows.

1. Try swapping out the ML model and play with hyperparameters and see if you can get a better accuracy
2. Try playing with the choice of "k" or the fragment length to see if you can improve the performance.
Does increasing "k" improve the performance?
Does this effect plateau or even reverse at some "k"?
3. Try using all the chromosomes of the cow genome as training data, and also downloading more mammalian and viral genomes to improve the diversity of the training dataset

